# Image Filtering
## Linear filtering
Correlation (in literature it is used as colvolution)
$$g(i, j) = \sum_{k}^{K} \sum_{l}^{L} I(i + k, j + l)*h(k, l)$$
* $I(x, y)$ is the image intensity at the pixel of corrdinates $(x, y)$
* $h(k, l)$ is the filter used which is called *kernel*
* $K$ is the kernel length
* $L$ is the kernel width
* $i, j$ is the *anchor*, the pixel that we are calculating the correlation for

### What should we do at the borders?
For example what should we do at pixel (0, 0)? There is not enough data around the pixel to apply the whole filter. We do one of the following:
For sake of simplicity imagine that a row in the image is `|1, 2, 3, 4|`
* Constant border: pad the borders of the image with some constant, can be zero, 255 or other, for example `1, 1|1, 2, 3, 4|1, 1`
* Replicate: pad the borders of the image with the last pixel value `1, 1|1, 2, 3, 4|4, 4`
* Reflect: pad the borders with a reflection of the row, `2, 1|1, 2, 3, 4|4, 3`
* Wrap around: `3, 4|1, 2, 3, 4|1, 2`
Check OpenCV border types https://docs.opencv.org/master/d2/de8/group__core__array.html#ga209f2f4869e304c82d07739337eae7c5

### Why linear filters are called linear?
Several computer vision applications are composed of step by step transformations of an input photo to output. This is easily done due to several properties associated with a common type of filters, that is, linear filters:

* The linear filters are commutative such that we can perform multiplication operations on filters in any order and the result still remains the same: $$a * b = b * a$$
* They are associative in nature, which means the order of applying the filter does not affect the outcome: $$(a * b) * c = a* (b * c)$$
* Even in cases of summing two filters, we can perform the first summation and then apply the filter, or we can also individually apply the filter and then sum the results. The overall outcome still remains the same:


In [49]:
from scipy.signal import convolve2d, correlate2d
import numpy as np
import cv2

In [50]:
import cv2
def visualize_image(figure_name: str, img: np.ndarray):
    cv2.imshow(figure_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [51]:
def show_images(images, titles):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    assert len(images) == len(titles)
    for title in titles:
        cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    
    for title, img in zip(titles, images):
        cv2.imshow(title, img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# How to use show_images([list of images], [list of titles]) They must have the same length
# show_images([img1, img2], ['This is image 1', 'This is image 2'])

In [84]:
from matplotlib import pyplot as plt
import cv2
#img = cv2.imread('/Desktop/LABSSSSSSSSSSSSSS/lab3-image filtering/assets/bird.jpg')
img = cv2.imread('C:/Users/HP 2019/Desktop/LABSSSSSSSSSSSSSS/lab3-image filtering/assets/original.png')
visualize_image('original', img)
grey_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#Box filtering is basically an average-of-surrounding-pixel kind of image filtering

### Box filter
This filter is the simplest of all! Each output pixel is the mean of its kernel neighbors ( all of them contribute with equal weights)
$$K = \frac{1}{K_{width} * K_{length}} * \begin{bmatrix} 1 & 1 & \cdots & 1\\ 1&1&\vdots&1 \\ 1 & 1 & \cdots & 1 \end{bmatrix}$$

It is used to filter the random noise in the image based on a simple idea, if one pixel has a random noise added then we can interpolate (average in our case) its value from the local neighbourhood.

What is the downside? it creates artifacts in the image! why? because it 

in OpenCV we can apply mean filter to image using `cv2.blur()` https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga8c45db9afe636703801b0b2e440fce37

In [12]:
# TODO: Use the blur function to test the effect of the box filter 
#cv2.blur() ----> Blurs an image using the normalized BOX  filter. 
# with different kernel sizes
# Visualize the original image with 3 blurred images at different sizes
#mean_blur_img1 = cv2.blur(img, ksize=(5, 5), borderType=cv2.BORDER_CONSTANT)
#visualize_image('Mean blur', mean_blur_img1)

#MY CODE 

#1-First blurring:
mean_blur_img1 = cv2.blur(img, ksize=(3, 3), borderType=cv2.BORDER_CONSTANT)
visualize_image('Mean blur_img1', mean_blur_img1)


#2-Second blurring:
mean_blur_img2 = cv2.blur(img, ksize=(5, 5), borderType=cv2.BORDER_CONSTANT)
visualize_image('mean_blur_img2', mean_blur_img2)

#3-Third blurring:
mean_blur_img3 = cv2.blur(img, ksize=(7, 7), borderType=cv2.BORDER_CONSTANT)
visualize_image('Mean blur_img3', mean_blur_img3)




### Gaussian filter
Probably one of the most useful filters (although not the fastest). Gaussian filtering is done by convolving each point in the input array with a *Gaussian kernel* and then summing them all to produce the output array.

$$G(x, y) = A e^{\frac{-(x - \mu_x)^2}{2 \sigma_x^2} + \frac{-(y - \mu_y)^2}{2 \sigma_y^2}}$$
* $G(x, y)$ is the Gaussian kernel
* $\mu$ is the mean 
* $\sigma^2$ is the variance 
The mean and variance are each per variable and we have 2 $(x, y)$

In OpenCV we can apply Gaussian filter to image using `cv2.GaussianBlur()`
https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1

`cv2.GaussianBlur()` has two main parameters
* ksize: Kernal size
* SigmaX, SigmaY: 
As a rule of thumb as stated in the lecture set filter half width to about $3\sigma$
To see the effect of changing the kernel size and sigma try it out
http://dev.theomader.com/gaussian-kernel-calculator/

In [13]:
import cv2
# TODO: Use the GaussianBlur function to test the blurring effect 
# with different kernel sizes and sigmas 
# Visualize the original image with 3 blurred images at different sizes
gaussian_blur_img1_with_sigma_1 = cv2.GaussianBlur(img, ksize=(3, 3), sigmaX=1, sigmaY=1, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img1_with_sigma_1', gaussian_blur_img1_with_sigma_1)
#increasing SIGMA value with same kernel size(3,3):
gaussian_blur_img1_with_sigma_3 = cv2.GaussianBlur(img, ksize=(3, 3), sigmaX=3, sigmaY=3, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img1_with_sigma_3', gaussian_blur_img1_with_sigma_3)

gaussian_blur_img1_with_sigma_6 = cv2.GaussianBlur(img, ksize=(3, 3), sigmaX=6, sigmaY=6, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img1_with_sigma_6', gaussian_blur_img1_with_sigma_6)
#.............................................
gaussian_blur_img2_with_sigma_1 = cv2.GaussianBlur(img, ksize=(5, 5), sigmaX=1, sigmaY=1, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img2_with_sigma_1', gaussian_blur_img2_with_sigma_1)
#increasing SIGMA value with same kernel size(5,5):

gaussian_blur_img2_with_sigma_3 = cv2.GaussianBlur(img, ksize=(5, 5), sigmaX=3, sigmaY=3, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img2_with_sigma_3', gaussian_blur_img2_with_sigma_3)
gaussian_blur_img2_with_sigma_6 = cv2.GaussianBlur(img, ksize=(5, 5), sigmaX=6, sigmaY=6, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img2_with_sigma_6', gaussian_blur_img2_with_sigma_6)
#..............................................................
gaussian_blur_img3_with_sigma_1 = cv2.GaussianBlur(img, ksize=(7, 7), sigmaX=1, sigmaY=1, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img3_with_sigma_1', gaussian_blur_img3_with_sigma_1)
#increasing SIGMA value with same kernel size(7,7):

gaussian_blur_img3_with_sigma_3 = cv2.GaussianBlur(img, ksize=(7, 7), sigmaX=3, sigmaY=3, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img3_with_sigma_3', gaussian_blur_img3_with_sigma_3)
gaussian_blur_img3_with_sigma_6 = cv2.GaussianBlur(img, ksize=(7, 7), sigmaX=6, sigmaY=6, borderType=cv2.BORDER_CONSTANT)
visualize_image('gaussian_blur_img3_with_sigma_6', gaussian_blur_img3_with_sigma_6)

#.................................................................
#Comparing image with smaller kernel size (3,3) and sigma (1,1) vs image with heigher kernel size(7,7) and sigma(6,6)
lower_blurring_image_vs_higher_blurring_image = cv2.hconcat((gaussian_blur_img1_with_sigma_1,gaussian_blur_img3_with_sigma_6))
name='comparing lower blurring vs higher blurring'

cv2.imshow(name,lower_blurring_image_vs_higher_blurring_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#..............................................................................
#Conclusion (Explanation) :

#Increasing sigma (variance) && kernel size values will increase blurring..because the distrubution will be accross 
#bigger part of image and hence more smoothing is applied .

#...............................................................................
#sources:Gaussian filter:
#https://www.youtube.com/watch?v=4RpdOAbnNYE
#https://www.youtube.com/watch?v=-AuwMJAqjJc

Now lets try to compute the correlation on our own using `correlate2d()`
https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.signal.correlate2d.html

In [14]:
kernel = np.ones((5, 5), dtype=np.uint8) * (1/25)
my_blur = np.array(img)
print('kernel shape is ',kernel.shape)
visualize_image('kernel figure', kernel)
my_blur[:,:,0] = correlate2d(img[:,:,0], kernel, 'same')
my_blur[:,:,1] = correlate2d(img[:,:,1], kernel, 'same')
my_blur[:,:,2] = correlate2d(img[:,:,2], kernel, 'same')
visualize_image('my blur', my_blur)

kernel shape is  (5, 5)


In [15]:
#TODO: Create function to create mean filter for an arbitary size
import math
import numpy as np
def create_mean_filter(ksize):
    # Remember to assert that the length is odd
    width=math.sqrt(ksize)
    width=int(width)
    height=width
    if(width%2==0):
        raise Exception('Error,filter dimensions should be odd !')
     
    constant=(1/(width*height))
    kernel=constant*(np.ones((width,height)))
    return kernel

In [16]:
######### MY CODE ######################

#To test our written mean filter :
import math
import numpy as np
from numpy import pi, exp, sqrt
#from skimage import io, img_as_ubyte, img_as_float32
#from skimage.transform import rescale
import cv2
import matplotlib.pyplot as plt

def conv(img: np.array, filter: np.array):  
    img_rows=img.shape[0]
    img_col=img.shape[1]
   
    filtered_image = np.zeros_like(img)
    #rotate filter 90 degrees horizontally and vertically 
    filter=np.rot90(filter,2)

    #To gurantee odd filters
    if(((filter.shape[0])%2)==0 or ((filter.shape[1])%2)==0):
      #print('Error,filter dimensions should be odd !')
      raise Exception('Error,filter dimensions should be odd !,output will be undefined ')
    filter_rows=filter.shape[0]
    filter_cols=filter.shape[1]
    padded_img=np.zeros((img_rows+filter_rows-1,img_col+filter_cols-1))
    #padded_img rows R and columns C 
    padded_img_R=padded_img.shape[0]
    padded_img_C=padded_img.shape[1]
    padded_img[((filter_rows)//2):padded_img_R-((filter_rows)//2),((filter_cols)//2):padded_img_C-((filter_cols)//2)]=img
    for i in range(((filter_rows)//2),padded_img_R-((filter_rows)//2)):
        for j in range(((filter_cols)//2),padded_img_C-((filter_cols)//2)):
            template=padded_img[i-((filter_rows)//2):(i+((filter_rows)//2)+1),j-((filter_cols)//2):(j+((filter_cols)//2)+1)]
            result=template*filter
            # element-wise multiplication filter && image
            filtered_image[i-((filter_rows)//2),j-((filter_cols)//2)]=result.sum()
    name='filtered image'  
    print('filtered_image',filtered_image.shape)
    #cv2_imshow(filtered_image)
    visualize_image(name,filtered_image)

    return filtered_image

In [36]:
# This function can be implemented using a double loop but since we 
# have numpy we can try it without loops 
# check np.meshgrid https://www.geeksforgeeks.org/numpy-meshgrid-function/import numpy as np
"""
Implement gaussian filter using for loop and meshgrid
"""
#MY CODE :

#Implement gaussain filter using for loop 
import math
def create_gaussian_filter(ksize, sigma):
    width=math.sqrt(ksize)
    width=int(width)
    height=width
    g=np.zeros((width,height))
    
    points_x = np.arange((-1*width//2)+1,(width//2)+1)   
    points_y=np.arange((-1*width//2)+1,(width//2)+1)
            
    for x in range(((-1*width//2)+1),((width//2)+1)):
        for y in range(((-1*height//2)+1),((height//2)+1)):
            g[x+(width//2),y+(height//2)]=(1/(2*math.pi*sigma*sigma))*math.exp((-1*(x*x+y*y))/(2*sigma*sigma))
    print('x array =',points_x)   
    print('y array =',points_y)                    
    return g

In [37]:
#MY CODE (Another way to implement gaussain filter using meshgrid) :
#meshgrid way:

import numpy as np
import math
def meshgrid_create_gaussian_filter(ksize, sigma):
    width=math.sqrt(ksize)
    width=int(width)
    height=width
    g=np.zeros((width,height))
    
    points_x = np.arange((-1*width//2)+1,(width//2)+1)   
    points_y=np.arange((-1*width//2)+1,(width//2)+1)
    x,y = np.meshgrid(points_x,points_y)
    print('x array =',points_x)
    print('y array =',points_y)
    print('x',x)
    print('y',y)
    
    constant=(1/(2*np.pi*sigma*sigma))
    g=constant*np.exp((-1*(x*x+y*y))/(2*sigma*sigma))
    return g
    
    
   

In [53]:
#MY CODE 
#To test create_gaussian_filter fn :
g_filter=create_gaussian_filter(25,3)

x array = [-2 -1  0  1  2]
y array = [-2 -1  0  1  2]


In [73]:
#MY CODE 
#To test meshgrid_create_gaussian_filter fn :
meshgrid_create_gaussian_filter(25,3)

x array = [-2 -1  0  1  2]
y array = [-2 -1  0  1  2]
x [[-2 -1  0  1  2]
 [-2 -1  0  1  2]
 [-2 -1  0  1  2]
 [-2 -1  0  1  2]
 [-2 -1  0  1  2]]
y [[-2 -2 -2 -2 -2]
 [-1 -1 -1 -1 -1]
 [ 0  0  0  0  0]
 [ 1  1  1  1  1]
 [ 2  2  2  2  2]]


array([[0.01133856, 0.01339492, 0.01416015, 0.01339492, 0.01133856],
       [0.01339492, 0.01582423, 0.01672824, 0.01582423, 0.01339492],
       [0.01416015, 0.01672824, 0.01768388, 0.01672824, 0.01416015],
       [0.01339492, 0.01582423, 0.01672824, 0.01582423, 0.01339492],
       [0.01133856, 0.01339492, 0.01416015, 0.01339492, 0.01133856]])

In [74]:
#MY CODE
#To test create_mean_filter && gaussian filter :

# create_mean_filte   fn :
mean_kernel=create_mean_filter(25)
conv(grey_image,mean_kernel)
print('kernel shape is ',mean_kernel.shape)   
print('mean kernel',mean_kernel)
print(type(mean_kernel))
#visualize_image('kernel figure', kernel)
#print('grey image',grey_image.shape)



#create_gaussian_filter   fn :
g_filter=create_gaussian_filter(25,3)
conv(grey_image,g_filter)

filtered_image (256, 256)
kernel shape is  (5, 5)
mean kernel [[0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]]
<class 'numpy.ndarray'>
x array = [-2 -1  0  1  2]
y array = [-2 -1  0  1  2]
filtered_image (256, 256)


array([[ 5.16307501,  6.88849563,  8.394176  , ...,  7.49378222,
         6.10153638,  4.50488764],
       [ 6.92691498,  9.26313212, 11.28054023, ..., 10.16192319,
         8.28134029,  6.11473972],
       [ 8.32270372, 11.19074221, 13.67133937, ..., 12.26094367,
        10.02062941,  7.40279046],
       ...,
       [ 0.47114235,  1.03375384, -0.21123774, ...,  1.16933471,
         1.15183177,  0.98446694],
       [ 0.95465619,  1.96942935,  0.90608848, ...,  2.17243735,
         1.65650177,  1.24423635],
       [ 0.78524188,  1.84226692,  1.21898912, ...,  1.92881573,
         1.39831974,  0.92585734]])

In [80]:
# TODO: Test your functions with `correlate2d()`
from scipy import signal
from scipy import misc
import cv2

######  MEAN FILTER  ###### 
correlated_mean = signal.correlate2d(grey_image,mean_kernel, boundary='symm', mode='same')
visualize_image('correlated imager by mean filter',correlated_mean)
######  GAUSSIAN FILTER  ###### 
#grey_image = grey_image - grey_image.mean()
#g_filter = g_filter - g_filter.mean()
correlated_gaussian = signal.correlate2d(grey_image, g_filter, boundary='fill', mode='same')
visualize_image('correlated imager by gaussian filter',correlated_gaussian)

# Frequency Domain

In [85]:
from scipy import fftpack

In [86]:
# This function maps matrices from frequency to space then plots them
def plot_image_from_freq(freq_domain_mat):
    inverse_fft_mat = fftpack.ifft2(freq_domain_mat) # Inverse FFT is a fast version of inverse DFT
    # Due to approximations, the returned matrix contains complex numbers
    # So, we get the magnitude to be able to plot the image
    image = np.abs(inverse_fft_mat) * 255
    visualize_image('ifft image', image)

In [101]:
# We will first try to construct a matrix in the frequency domain that makes a vertically moving ripple in the space domain

freq_domain_mat = np.zeros([21,21])
freq_domain_mat[9, 10] = 1 # The choice of the value '1' is arbitrary
freq_domain_mat[11, 10] = 1
plot_image_from_freq(freq_domain_mat)

In [102]:
# TODO: construct a matrix in the frequency domain that makes a HORIZONTALLY moving ripple in the space domain

freq_domain_mat = np.zeros([21,21])
freq_domain_mat[11, 11] = 1
freq_domain_mat[9, 9] = 1
plot_image_from_freq(freq_domain_mat)

In [103]:
#MY CODE 
#SOLUTION :
freq_domain_mat = np.zeros([21,21])
freq_domain_mat[10, 9] = 1 # The choice of the value '1' is arbitrary
freq_domain_mat[10, 11] = 1
plot_image_from_freq(freq_domain_mat)

In [104]:
# This function applies a filter to an image in the frequency domain
# and plots multiple images describing the process
def apply_filter_in_freq(img, f):
    img_in_freq = fftpack.fft2(img)
    
    # we supply the img shape here to make both the filter and img have the same shape to be able to multiply
    filter_in_freq = fftpack.fft2(f, img.shape)
    filtered_img_in_freq = np.multiply(img_in_freq, filter_in_freq)
    filtered_img = fftpack.ifft2(filtered_img_in_freq)
    
    img_in_freq_domain = fftpack.fftshift(np.log(np.abs(img_in_freq)+1))
    img_in_freq_domain /= img_in_freq_domain.max() - img_in_freq_domain.min()
    
    filter_in_freq_domain = fftpack.fftshift(np.log(np.abs(filter_in_freq)+1))
    filter_in_freq_domain /= filter_in_freq_domain.max() - filter_in_freq_domain.min()
    
    filtered_img_in_freq_domain = fftpack.fftshift(np.log(np.abs(filtered_img_in_freq)+1))
    filtered_img_in_freq_domain /= filtered_img_in_freq_domain.max() - filtered_img_in_freq_domain.min()
    
    filtered_img = np.abs(filtered_img)
    filtered_img = (filtered_img - filtered_img.min()) / (filtered_img.max() - filtered_img.min())
    
    show_images([img,
                img_in_freq_domain, # log for better intensity scale, 
                filter_in_freq_domain,
                filtered_img_in_freq_domain,
                filtered_img
                ], ['Image', 'Image in Freq. Domain', 'Filter in Freq. Domain', 'Filtered Image in Freq. Domain', 'Filtered Image'])

In [105]:
img2 = cv2.imread('./assets/Picture2.png')
gray_img = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) / 255
gray_img = (gray_img - gray_img.min()) / (gray_img.max() - gray_img.min())

# This is a low pass filter
f=np.array([
    [1,1,1],
    [1,1,1],
    [1,1,1]
], dtype=np.float)

f *= 1/9

apply_filter_in_freq(gray_img, f)

In [106]:
# This is a low pass filter
f=np.array([
    [1,2,1],
    [2,4,2],
    [1,2,1]
])

apply_filter_in_freq(gray_img, f)

In [107]:
# This is a high pass filter 
f=np.array([
    [ 0,-1, 0],
    [-1, 4,-1],
    [ 0,-1, 0]
])

apply_filter_in_freq(gray_img, f)